In [14]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
import xgboost as xgb

In [15]:
def get_data_loader(file, features, features_to_encode):
    df = pd.read_csv(file)
    df["Episode_Title"] = df["Episode_Title"].str[8:]
    y = df[["Listening_Time_minutes"]]
    df = df[features]
    imputer = SimpleImputer(strategy="median")
    df[["Episode_Length_minutes"]] = imputer.fit_transform(df[["Episode_Length_minutes"]])
    df[["Guest_Popularity_percentage"]] = imputer.fit_transform(df[["Guest_Popularity_percentage"]])
    df[["Number_of_Ads"]] = imputer.fit_transform(df[["Number_of_Ads"]])
    df["Is_Weekend"] = (df["Publication_Day"] == "Saturday") | (df["Publication_Day"] == "Sunday")
    def one_hot(df, feature):
        encoded = pd.get_dummies(df[[feature]])
        result = pd.concat([df, encoded], axis=1)
        result = result.drop([feature], axis=1)
        return(result) 
    for to_encode in features_to_encode:
        df = one_hot(df, to_encode)
    df["Ads_Per_Minute"] = df['Number_of_Ads'] / df["Episode_Length_minutes"]
    df["People_Popularity"] = df["Host_Popularity_percentage"] + df["Guest_Popularity_percentage"]
    df["Linear"] = df["Episode_Length_minutes"] * 0.728

    x = df
    x = x.to_numpy().astype(np.float32)
    y = y.to_numpy().astype(np.float32)
    return x, y

features=["Episode_Length_minutes", "Number_of_Ads", "Host_Popularity_percentage", "Guest_Popularity_percentage",
          "Episode_Sentiment", "Publication_Day", "Publication_Time", "Genre", "Podcast_Name"]
features_to_encode = ["Episode_Sentiment", "Publication_Day", "Publication_Time",  "Genre", "Podcast_Name"]
x, y = get_data_loader("train.csv", 
    features=features,
    features_to_encode=features_to_encode )

KeyboardInterrupt: 

In [ ]:
'''
lr = 0.375
n_estimators = 1200
max_bin = 1024

common_params = {
    "objective": "reg:squarederror",
    "n_estimators": n_estimators,
    "max_depth": 6,
    "learning_rate": lr,
    "max_bin": max_bin,
    "verbosity": 0
}

models = []
for _ in range(10):
    model = xgb.XGBRegressor(**common_params)
    model.fit(x, y, eval_set=[(x, y)], verbose=True)
    models.append(model)

model1, model2, model3, model4, model5, model6, model7, model8, model9, model10 = models
'''


[0]	validation_0-rmse:19.88157
[1]	validation_0-rmse:16.16169
[2]	validation_0-rmse:14.43955
[3]	validation_0-rmse:13.69922
[4]	validation_0-rmse:13.38991
[5]	validation_0-rmse:13.25823
[6]	validation_0-rmse:13.19888
[7]	validation_0-rmse:13.17082
[8]	validation_0-rmse:13.15506
[9]	validation_0-rmse:13.14579
[10]	validation_0-rmse:13.13717
[11]	validation_0-rmse:13.13075
[12]	validation_0-rmse:13.12474
[13]	validation_0-rmse:13.11599
[14]	validation_0-rmse:13.11073
[15]	validation_0-rmse:13.10784
[16]	validation_0-rmse:13.10334
[17]	validation_0-rmse:13.09985
[18]	validation_0-rmse:13.09485
[19]	validation_0-rmse:13.09064
[20]	validation_0-rmse:13.08800
[21]	validation_0-rmse:13.08205
[22]	validation_0-rmse:13.07873
[23]	validation_0-rmse:13.07552
[24]	validation_0-rmse:13.07128
[25]	validation_0-rmse:13.06610
[26]	validation_0-rmse:13.06370
[27]	validation_0-rmse:13.06134
[28]	validation_0-rmse:13.05737
[29]	validation_0-rmse:13.05342
[30]	validation_0-rmse:13.04962
[31]	validation_0-

In [ ]:
from sklearn.model_selection import KFold
import lightgbm as lgb

# Hyperparameters
lr = 0.03
n_estimators = 1000
max_bin = 1024
num_leaves = 1024
max_depth = 1
subsample = 0.7
colsample_bytree = 0.7

# Common parameters
common_params = {
    "objective": "regression",
    "n_estimators": n_estimators,
    "learning_rate": lr,
    "max_bin": max_bin,
    "num_leaves": num_leaves,
    "max_depth": max_depth,
    "subsample": subsample,
    "colsample_bytree": colsample_bytree,
}

kf = KFold(n_splits=10, shuffle=True, random_state=42)
models = []

for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model = lgb.LGBMRegressor(**common_params)
    model.fit(x_train, y_train, eval_set=[(x_val, y_val)])
    models.append(model)



c:\repos\class\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\repos\class\.venv\Lib\site-packages\lightgbm\basic.py:374: UserWarning: Converting column-vector to 1d array
  _log_warning("Converting column-vector to 1d array")
c:\repos\class\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\repos\class\.venv\Lib\site-packages\lightgbm\basic.py:374: UserWarning: Converting column-vector to 1d array
  _log_warning("Converting column-vector to 1d array")
c:\repos\class\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array w

In [ ]:
def get_test_loader(file, features, features_to_encode):
    df = pd.read_csv(file)
    def one_hot(df, feature):
        encoded = pd.get_dummies(df[[feature]])
        result = pd.concat([df, encoded], axis=1)
        result = result.drop([feature], axis=1)
        return(result) 
    df["Episode_Title"] = df["Episode_Title"].str[8:]
    df = df[features]
    imputer = SimpleImputer(strategy="median")
    df[["Episode_Length_minutes"]] = imputer.fit_transform(df[["Episode_Length_minutes"]])
    df[["Guest_Popularity_percentage"]] = imputer.fit_transform(df[["Guest_Popularity_percentage"]])
    df[["Number_of_Ads"]] = imputer.fit_transform(df[["Number_of_Ads"]])
    df["Is_Weekend"] = (df["Publication_Day"] == "Saturday") | (df["Publication_Day"] == "Sunday")
    for to_encode in features_to_encode:
        df = one_hot(df, to_encode)
    df["Ads_Per_Minute"] = df['Number_of_Ads'] / df["Episode_Length_minutes"]
    df["People_Popularity"] = df["Host_Popularity_percentage"] + df["Guest_Popularity_percentage"]
    df["Linear"] = df["Episode_Length_minutes"] * 0.728
    x = df
    x_tensor = x.to_numpy().astype(np.float32)
    return x_tensor



features=["Episode_Length_minutes", "Number_of_Ads", "Host_Popularity_percentage", "Guest_Popularity_percentage",
          "Episode_Sentiment", "Publication_Day", "Publication_Time", "Genre", "Podcast_Name" ]
features_to_encode = ["Episode_Sentiment", "Publication_Day", "Publication_Time",  "Genre", "Podcast_Name"]
test_dataloader = get_test_loader("test.csv", 
    features=features,
    features_to_encode=features_to_encode )

X_test = get_test_loader("test.csv", features, features_to_encode)


preds = sum(model.predict(X_test) for model in models) / len(models)
# Write to submission file
import csv

with open("submission_lgbm.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["id", "prediction"])
    for i, pred in enumerate(preds, start=750000):
        writer.writerow([i, pred])


c:\repos\class\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\repos\class\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\repos\class\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\repos\class\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\repos\class\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\repos\class\.venv\Lib\site-packages\sklearn\utils\validat